In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import json
import numpy as np
from torch.optim.lr_scheduler import CosineAnnealingLR
from pose_dataloader import PoseDataset  # 导入你定义的数据集类
from mlp_classifier import MLPClassifier  # 导入你定义的模型

In [2]:
# 数据集路径
input_json_path = "../data/SyRIP_Posture/annotations/train600/processed_train_data.json"
test_json_path = "../data/SyRIP_Posture/annotations/validate100/processed_validate_data.json"  # 假设你有测试数据集


In [3]:

# 创建数据加载器
train_dataset = PoseDataset(input_json_path)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 测试集加载器
test_dataset = PoseDataset(test_json_path)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 设置输入尺寸
input_size = 34  # 17个关键点，每个关键点2个坐标 -> 17*2 = 34
hidden_size = 256  # 隐藏层大小
output_size = 4  # 假设有4个姿势（Sitting, Supine, Prone, Standing）

In [4]:
# 模型定义
model = MLPClassifier(input_size, hidden_size, output_size)

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数

# 优化器参数调整：较小的学习率、weight_decay（L2 正则化）
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)  # 调整学习率和 L2 正则化强度

# 学习率调度器：CosineAnnealingLR
scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-6)  # 每100个epoch减小学习率


In [5]:
# 训练函数
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 清空梯度
            optimizer.zero_grad()

            # 正向传播
            outputs = model(inputs)

            # 计算损失
            loss = criterion(outputs, labels)
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数

            running_loss += loss.item()

        # 更新学习率
        scheduler.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# 评估函数
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 正向传播
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy}%')
    return accuracy

In [6]:
# 主程序入口
if __name__ == "__main__":
    # 训练模型
    train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=1000)

    # 保存模型
    torch.save(model.state_dict(), "pose_classifier.pth")

    # 测试评估
    evaluate_model(model, test_loader)

Epoch [1/1000], Loss: 1.3463354612651623
Epoch [2/1000], Loss: 1.2370956132286473
Epoch [3/1000], Loss: 1.1780410063894171
Epoch [4/1000], Loss: 1.1503170477716547
Epoch [5/1000], Loss: 1.1132168832578158
Epoch [6/1000], Loss: 1.0946643415250277
Epoch [7/1000], Loss: 1.079393091954683
Epoch [8/1000], Loss: 1.0555900084344965
Epoch [9/1000], Loss: 1.0763375288561772
Epoch [10/1000], Loss: 1.0669790945555035
Epoch [11/1000], Loss: 1.0580843059640181
Epoch [12/1000], Loss: 1.0439542375112836
Epoch [13/1000], Loss: 1.0456422630109286
Epoch [14/1000], Loss: 1.0314580327586125
Epoch [15/1000], Loss: 1.0430136291604293
Epoch [16/1000], Loss: 1.0424763560295105
Epoch [17/1000], Loss: 1.0247183222519725
Epoch [18/1000], Loss: 1.0373028861848932
Epoch [19/1000], Loss: 1.0375059623467295
Epoch [20/1000], Loss: 1.0019852173955817
Epoch [21/1000], Loss: 1.017106360510776
Epoch [22/1000], Loss: 1.0157758311221474
Epoch [23/1000], Loss: 1.0255900903751975
Epoch [24/1000], Loss: 1.0066021869057102
Epo